# The Django template language

## Templates

> A template is a text file. It can generate any text-based format (HTML, XML, CSV, etc.).

ด้านล่างเป็นตัวอย่าง template ที่มีการใช้งานทุกๆ concept ที่เราจะพูดถึงในวันนี้

In [ ]:
{% extends "base_generic.html" %}

{% block title %}{{ section.title }}{% endblock %}

{% block content %}
<h1>{{ section.title }}</h1>

{% for story in story_list %}
<h2>
  <a href="{{ story.get_absolute_url }}">
    {{ story.headline|upper }}
  </a>
</h2>
<p>{{ story.tease|truncatewords:"100" }}</p>
{% endfor %}
{% endblock %}

## Variables

การแสดงค่าตัวแปรใน tamplate จะใช้ {{ variable }}

โดยถ้าเราพยายามแสดงค่าตัวแปรที่ไม่มีการส่งมาใน template ค่า '' จะถูกแสดงแทน

## Filters

Filters ใช้ในการ modify ค่าของตัวแปร เช่น {{ name|lower }} โดย `|` จะเป็น syntax การ apply filter เข้าไปกับตัวแปร

เราสามารถ chain filter หลายๆตัวได้ เช่น {{ text|escape|linebreaks }}

นอกจากนั้น filter บางตัวก็มีการรับ argument เช่น {{ bio|truncatewords:30 }}

[built-in filter reference](https://docs.djangoproject.com/en/5.2/ref/templates/builtins/#ref-templates-builtins-filters)

Filter ที่ใช้บ่อยๆ:

In [ ]:
{{ value|default:"nothing" }}

{{ value|length }}

{{ value|filesizeformat }}
<!--If value is 123456789, the output would be 117.7 MB.-->

## Tags

Syntax ของ tag จะเป็น {% %} โดย tag จะเกี่ยวข้องกับ control flow และ logic

และบาง tag จะต้องมีการเปิด และ ปิด tag

[built-in tag reference](https://docs.djangoproject.com/en/5.2/ref/templates/builtins/#ref-templates-builtins-tags)

เช่น

**for**

In [ ]:
<ul>
{% for athlete in athlete_list %}
    <li>{{ athlete.name }}</li>
{% endfor %}
</ul>

**if elif else**

In [ ]:
{% if athlete_list %}
    Number of athletes: {{ athlete_list|length }}
{% elif athlete_in_locker_room_list %}
    Athletes should be out of the locker room soon!
{% else %}
    No athletes.
{% endif %}

## Comments

In [ ]:
{# greeting #}hello

{# {% if foo %}bar{% else %} #}

## Template inheritance

Template inheritance นั้นช่วยให้เราสามารถวางโครงสร้างของหน้าเพจ โดยการสร้าง base "skeleton" เช่น

![layout](./layout.png)

ลองมาดูตัวอย่่างการทำ template inheritance กัน

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <link rel="stylesheet" href="style.css">
    <title>{% block title %}My amazing site{% endblock %}</title>
</head>

<body>
    <div id="sidebar">
        {% block sidebar %}
        <ul>
            <li><a href="/">Home</a></li>
            <li><a href="/blog/">Blog</a></li>
        </ul>
        {% endblock %}
    </div>

    <div id="content">
        {% block content %}{% endblock %}
    </div>
</body>
</html>

ไฟล์ `base.html` นี้ทำการกำหนดโครงสร้างของหน้าเวบเพจเป็น 3 ส่วนคือ

1. title
2. sidebar
3. content

โดยการใช้ tag **block**

หน้าเพจที่มา extend `base.html` นี้ไปอาจจะเป็นดังตัวอย่าง

In [ ]:
{% extends "base.html" %}

{% block title %}My amazing blog{% endblock %}

{% block content %}
{% for entry in blog_entries %}
    <h2>{{ entry.title }}</h2>
    <p>{{ entry.body }}</p>
{% endfor %}
{% endblock %}

จะเห็นได้ว่ามีการใช้ tag **extends** - `{% extends "base.html" %}`

หน้า "My amazing blog" นี้จะถูก Django render เป็น HTML ออกมาดังนี้

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <link rel="stylesheet" href="style.css">
    <title>My amazing blog</title>
</head>

<body>
    <div id="sidebar">
        <ul>
            <li><a href="/">Home</a></li>
            <li><a href="/blog/">Blog</a></li>
        </ul>
    </div>

    <div id="content">
        <h2>Entry one</h2>
        <p>This is my first entry.</p>

        <h2>Entry two</h2>
        <p>This is my second entry.</p>
    </div>
</body>
</html>

จะเห็นว่าตัวลูกที่ extend `base.html` ไม่ได้ทำการกำหนด block "sidebar" ดังนั้น code ใน block นั้นจะเป็นตามค่าที่กำหนดใน `base.html`

## Automatic HTML escaping

เมื่อเราทำการ render HTML จากข้อมูลที่ส่งมาจาก database มันมักจะมีความเสี่ยงที่ข้อมูลจะทำให้หน้าเพจ HTML เกิดปัญหา เช่น

In [ ]:
Hello, {{ name }}

--------------

<!-- ถ้าค่าของตัวแปร name คือ -->
<script>alert('hello')</script>

-------------

<!-- เราก็จะ render ได้ -->
Hello, <script>alert('hello')</script>

หรือค่าของ name = <b>username ก็จะทำให้หน้า page ทั้งหมดต่อจากตรงนี้เป็น bold ทันที

ดังนั้นโดย default Django จะทำการ escape ค่าของตัวแปรที่เอามา render ใน templage ให้อัตโนมัติ

- `<` is converted to `&lt;`
- `>` is converted to `&gt;`
- `'` (single quote) is converted to `&#x27;`
- `"` (double quote) is converted to `&quot;`
- `&` is converted to `&amp;`

### How to turn it off

เราสามารถปิด auto-escape ได้หลายระดับ ทั้ง per-site, per-template level หรือ per-variable level

#### Variable level

In [ ]:
This will be escaped: {{ data }}
This will not be escaped: {{ data|safe }}

-------

This will be escaped: &lt;b&gt;
This will not be escaped: <b>

#### Template block level

In [ ]:
Auto-escaping is on by default. Hello {{ name }}

{% autoescape off %}
    This will not be auto-escaped: {{ data }}.

    Nor this: {{ other_data }}
    {% autoescape on %}
        Auto-escaping applies again: {{ name }}
    {% endautoescape %}
{% endautoescape %}

## Accessing method calls

เราสามารถเรียก function ของ instance ที่เราส่งไป render ที่ template ได้ เช่น

In [ ]:
{% for comment in task.comment_set.all %}
    {{ comment }}
{% endfor %}

-------

{{ task.comment_set.all.count }}

หรือถ้าเรามีการ define method ของ class เอาไว้ เราก็สามารถเรียก method ของ instance ของ class นั้นได้ใน template

In [ ]:
<!-- in models.py -->
class Task(models.Model):
    def foo(self):
        return "bar"

-------

<!-- in template.html -->
{{ task.foo }}

**หมายเหตุ: แต่เราจะไม่สามารถส่ง argument เข้า function ได้นะครับใน template**

## Custom tag and filter libraries

เราสามารถสร้าง customer tag และ filter ได้เอง ละก็มีคนอื่นที่เขาทำ libraries ของ third-party custom tags and filters มาให้ใช้งานด้วยเช่น `humanize`

วิธีติดตั้ง `humanize`:

1. pip install humanize
2. เพิ่ม 'django.contrib.humanize' ใน INSTALLED_APPS ในไฟล์ settings

เช่น

In [ ]:
{% load humanize %}

{{ 45000|intcomma }}

## How to create custom template tags and filters

การสร้าง custom template tags และ filters เริ่มด้วยการสร้าง folder `templatetags` ใน app ที่เกี่ยวข้อง (สร้างไว้ระดับเดียวกับ views.py/models.py) 

สมมติเราจะสร้าง template tag ชื่อ poll_extras

```
polls/
    __init__.py
    models.py
    templatetags/
        __init__.py
        poll_extras.py
    views.py
```

การนำไปใช้งานใน template

```
{% load poll_extras %}
```

### Writing custom template filters

Custom filters ก็คือ Python function ที่รับ 1 หรือ 2 arguments
- ค่าตัวแปร input
- ค่าตัวแปร argument ซึ่งอาจจะเป็นค่า default หรือไม่มีเลยก็ได้

เช่น

In [ ]:
%%python
def cut(value, arg):
    """Removes all values of arg from the given string"""
    return value.replace(arg, "")

การเรียกใช้ใน template

```
{% load custom_filters %}

{{ somevariable|cut:"0" }}
```

โดยถ้าจะไม่รับ argument ก็ได้ ตอนประกาศ function ก็ไม่ต้องใส่

In [ ]:
%%python
def lower(value):  # Only one argument.
    """Converts a string into all lowercase"""
    return value.lower()

#### Registering custom filters

สำหรับการ register ให้ Django รู้จัก custom filter ที่เราสร้างขึ้นมาสามารถทำได้โดยใช้ `register.filter()`

`register.filter()` รับ argument 2 ตัวได้แก่
- ชื่อของ filter สำหรับใช้เรียกใน template - string
- ตัว Python function ที่เราสร้างขึ้นมา

In [ ]:
%%python
from django import template

register = template.Library()

def cut(value, arg):
    """Removes all values of arg from the given string"""
    return value.replace(arg, "")

def lower(value):  # Only one argument.
    """Converts a string into all lowercase"""
    return value.lower()

register.filter("cut", cut)
register.filter("lower", lower)

หรือ register โดยใช้ decorator

In [ ]:
%%python
@register.filter(name="cut")
def cut(value, arg):
    return value.replace(arg, "")


@register.filter
def lower(value):
    return value.lower()

### Writing custom template tags

Tag จะมีความซับซ้อนมากกว่า filter เพราะเราจะทำอะไรก็ได้ โดยเราจะใช้ `library.simple_tag()` ในการ register ตัว function

ยกตัวอย่างเช่นถ้าเราอยากจะเขียน template tag ที่แสดง วันที่-เวลา ปัจจุบัน

In [ ]:
%%python
import datetime
from django import template

register = template.Library()

@register.simple_tag
def current_time(format_string):
    return datetime.datetime.now().strftime(format_string)

# กำหนดชื่อ template tag
@register.simple_tag(name="minustwo")
def some_function(value):
    return value - 2

# สามารถรับ arguments กี่ตัวก็ได้
@register.simple_tag
def my_tag(a, b, *args, **kwargs):
    warning = kwargs["warning"]
    profile = kwargs["profile"]
    ...
    return ...

การเรียกใช้ใน template

```
{% load custom_tags %}

{% my_tag 123 "abcd" book.title warning=message|lower profile=user.profile %}
```

สามารถเก็บค่าที่ได้รับจาก template tag ในตัวแปรได้

```
{% current_time "%Y-%m-%d %I:%M %p" as the_time %}
<p>The time is {{ the_time }}.</p>
```
